<a href="https://colab.research.google.com/github/bourydamien/projet_scoring/blob/master/scoring_colab_catboost_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation des requirements stockés sur mon dépot git

In [1]:
import subprocess

def install_requirements_from_github(url):
    # Télécharger le fichier requirements.txt depuis GitHub
    subprocess.run(['curl', '-o', 'requirements.txt', url])

    # Installer les bibliothèques listées dans requirements.txt
    subprocess.run(['pip', 'install', '-r', 'requirements.txt'])

# URL du fichier requirements.txt sur GitHub
requirements_url = 'https://raw.githubusercontent.com/bourydamien/projet_scoring/master/requirements.txt'

# Appeler la fonction pour installer les bibliothèques
install_requirements_from_github(requirements_url)



imports des fonctions installées

In [2]:
import numpy as np
import pandas as pd
import glob
import optuna

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier, Pool, utils

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import mlflow
import mlflow.sklearn
import gc
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import gdown
import numpy as np
import pandas as pd
from scipy.stats import boxcox
from scipy.stats import skew
from google.colab import drive
import catboost
from catboost.utils import get_gpu_device_count
import os
import pickle
import joblib
from datetime import datetime
import json


from catboost.utils import get_gpu_device_count

Chemin ou je dois enregistrer les modeles sur le drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

scoring_folder = '/content/drive/MyDrive/Colab Notebooks/data_scoring'

# Créer un sous-dossier avec la date et l'heure
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_folder = f'{scoring_folder}/model_{timestamp}'
os.makedirs(model_folder, exist_ok=True)


chargement des fichiers d'entrainements et de test. ( Pour plus de facilité, je les aient stockés sur le drive google)

In [5]:
def download_data():
    # URLs des fichiers sur Google Drive
    train_url = 'https://drive.google.com/uc?id=1oElrtY8GECJGLFpcB8-P5H5nwTvfhcYk'
    test_url = 'https://drive.google.com/uc?id=1OnzEHhlJqNASnP1FHiHPnb2554agP9cZ'

    # Chemins de sauvegarde locaux
    train_output = 'train_data.csv'
    test_output = 'test_data.csv'

    # Télécharger les fichiers
    gdown.download(train_url, train_output, quiet=False)
    gdown.download(test_url, test_output, quiet=False)

# Appeler la fonction pour télécharger les données
download_data()


Downloading...
From (original): https://drive.google.com/uc?id=1oElrtY8GECJGLFpcB8-P5H5nwTvfhcYk
From (redirected): https://drive.google.com/uc?id=1oElrtY8GECJGLFpcB8-P5H5nwTvfhcYk&confirm=t&uuid=5034e1a1-6f3d-4309-a477-f6057308b005
To: /content/train_data.csv
100%|██████████| 166M/166M [00:04<00:00, 41.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OnzEHhlJqNASnP1FHiHPnb2554agP9cZ
To: /content/test_data.csv
100%|██████████| 26.6M/26.6M [00:00<00:00, 32.1MB/s]


Chargement des données en dataframe

In [6]:
#importation des données de train
app_train = pd.read_csv('/content/train_data.csv')
print('Training data shape: ', app_train.shape)
# Testing data features
app_test = pd.read_csv('/content/test_data.csv')
print('Testing data shape: ', app_test.shape)


Training data shape:  (307511, 122)
Testing data shape:  (48744, 121)


Feature engeneering et correction des anomalies

In [7]:
def add_domain_features_with_anomaly(train_df, test_df):
    """
    Ajoute une colonne d'anomalie pour 'DAYS_EMPLOYED' et remplace les valeurs anormales,
    puis applique les transformations sur les DataFrames de train et de test.

    Args:
        train_df (pd.DataFrame): Le DataFrame d'entraînement.
        test_df (pd.DataFrame): Le DataFrame de test.

    Returns:
        train_df (pd.DataFrame): Le DataFrame d'entraînement avec les modifications.
        test_df (pd.DataFrame): Le DataFrame de test avec les modifications.
    """

    # Ajouter une colonne 'DAYS_EMPLOYED_ANOM' (anomalie) et remplacer 365243 par NaN
    for df in [train_df, test_df]:
        df['DAYS_EMPLOYED_ANOM'] = df["DAYS_EMPLOYED"] == 365243
        df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

        # Ajouter les colonnes dérivées
        df['CREDIT_INCOME_PERCENT'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
        df['ANNUITY_INCOME_PERCENT'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
        df['CREDIT_TERM'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
        df['DAYS_EMPLOYED_PERCENT'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    print("Nouvelles colonnes ajoutées et anomalies corrigées")

    return train_df, test_df

In [8]:
app_train, app_test = add_domain_features_with_anomaly(app_train,app_test)

Nouvelles colonnes ajoutées et anomalies corrigées


In [9]:
set(app_train.columns)-set(app_test.columns)

{'TARGET'}

Encodage des variables catégorielles

In [10]:
def encode_categorical_variables(train_data, test_data, target_column, model_folder):
    """
    Encode les variables catégorielles en deux étapes :
    1. Utilise Label Encoding pour les colonnes avec 2 catégories ou moins.
    2. Utilise One-Hot Encoding pour toutes les autres variables catégorielles.
    Puis aligne les colonnes des jeux de données et réinsère la variable cible.

    Les objets utilisés pour l'encodage (LabelEncoder, colonnes One-Hot) sont enregistrés dans `model_folder`.

    :param train_data: DataFrame des données d'entraînement
    :param test_data: DataFrame des données de test
    :param target_column: Nom de la colonne cible dans les données d'entraînement
    :param model_folder: Dossier où enregistrer les objets d'encodage
    :return: DataFrames transformés (train_data, test_data)
    """

    # Extraire la variable cible
    train_labels = train_data[target_column]

    # Initialiser le label encoder
    le = LabelEncoder()
    le_count = 0
    label_encoded_columns = []

    # Appliquer le Label Encoding pour les colonnes avec 2 catégories ou moins
    for col in train_data:
        if train_data[col].dtype == 'object':
            # Si 2 catégories ou moins
            if len(list(train_data[col].unique())) <= 2:
                # Entraîner le label encoder sur les données d'entraînement
                le.fit(train_data[col])
                # Transformer les colonnes d'entraînement et de test
                train_data[col] = le.transform(train_data[col])
                test_data[col] = le.transform(test_data[col])

                le_count += 1
                label_encoded_columns.append(col)

    print(f'{le_count} colonnes ont été label encodées.')

    # Si des colonnes ont été label encodées, enregistrer le LabelEncoder
    if label_encoded_columns:
        with open(os.path.join(model_folder, 'label_encoder.pkl'), 'wb') as f:
            pickle.dump(le, f)

        # Enregistrer la liste des colonnes label encodées
        with open(os.path.join(model_folder, 'label_encoded_columns.pkl'), 'wb') as f:
            pickle.dump(label_encoded_columns, f)

    # Appliquer le One-Hot Encoding sur toutes les autres colonnes catégorielles
    train_data = pd.get_dummies(train_data)
    test_data = pd.get_dummies(test_data)

    # Enregistrer les colonnes utilisées pour le One-Hot Encoding
    with open(os.path.join(model_folder, 'one_hot_columns.pkl'), 'wb') as f:
        pickle.dump(train_data.columns.tolist(), f)

    # Aligner les colonnes des jeux de données d'entraînement et de test
    train_data, test_data = train_data.align(test_data, join='inner', axis=1)

    # Réinsérer la colonne cible dans le jeu d'entraînement
    train_data[target_column] = train_labels

    print('Forme des données d\'entraînement après One-Hot Encoding et alignement: ', train_data.shape)
    print('Forme des données de test après One-Hot Encoding et alignement: ', test_data.shape)

    return train_data, test_data



In [11]:
app_train , app_test = encode_categorical_variables(app_train, app_test, "TARGET", model_folder)

3 colonnes ont été label encodées.
Forme des données d'entraînement après One-Hot Encoding et alignement:  (307511, 245)
Forme des données de test après One-Hot Encoding et alignement:  (48744, 244)


Transformation mathématique des colonnes avec un boxcox si les données sont asymétriques

In [12]:
def apply_boxcox_transformation(train_data, test_data, skew_threshold=0.75, save_path=model_folder):
    """
    Applique la transformation Box-Cox sur les colonnes numériques du DataFrame qui sont asymétriques (skewed)
    dans les jeux d'entraînement et de test, et enregistre les informations utiles (colonnes transformées et lambdas).

    :param train_data: DataFrame d'entraînement
    :param test_data: DataFrame de test
    :param skew_threshold: Seuil de skewness pour décider de l'application de la transformation Box-Cox
    :param save_path: Chemin où enregistrer les informations du modèle (colonnes et lambdas)
    :return: DataFrames transformés (train_data, test_data) et liste des colonnes transformées avec lambda
    """

    # Sélectionner les colonnes numériques dans les deux datasets
    numeric_cols = train_data.select_dtypes(include=[np.number]).columns

    # Initialiser une liste pour garder la trace des colonnes transformées
    transformed_cols = []
    lambda_params = {}

    # Appliquer la transformation sur chaque colonne numérique
    for col in numeric_cols:
        # Vérifier que toutes les valeurs dans les deux jeux de données sont strictement positives
        if (train_data[col] > 0).all() and (test_data[col] > 0).all():
            # Calculer la skewness sur le jeu d'entraînement
            col_skew = skew(train_data[col])

            # Appliquer Box-Cox si l'asymétrie dépasse le seuil
            if abs(col_skew) > skew_threshold:
                # Appliquer la transformation Box-Cox sur le jeu d'entraînement
                train_data[col], fitted_lambda = boxcox(train_data[col])

                # Appliquer la transformation Box-Cox sur le jeu de test avec le même lambda
                test_data[col] = boxcox(test_data[col], lmbda=fitted_lambda)

                # Ajouter la colonne transformée et le lambda à la liste
                transformed_cols.append(col)
                lambda_params[col] = fitted_lambda

    # Afficher les colonnes transformées et les paramètres lambda utilisés
    print(f"Colonnes transformées avec Box-Cox : {transformed_cols}")
    print(f"Paramètres lambda utilisés : {lambda_params}")

    # Sauvegarder les colonnes transformées et les paramètres lambda dans des fichiers JSON
    with open(f'{save_path}/transformed_columns.json', 'w') as f:
        json.dump(transformed_cols, f)

    with open(f'{save_path}/lambda_params.json', 'w') as f:
        json.dump(lambda_params, f)

    return train_data, test_data, transformed_cols, lambda_params

In [13]:
train_data, test_data, transformed_cols, lambda_params = apply_boxcox_transformation(app_train,app_test, save_path=model_folder)

Colonnes transformées avec Box-Cox : ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'REGION_POPULATION_RELATIVE', 'CREDIT_INCOME_PERCENT']
Paramètres lambda utilisés : {'AMT_INCOME_TOTAL': -0.09313337743754035, 'AMT_CREDIT': 0.1920818791540177, 'REGION_POPULATION_RELATIVE': 0.26957000753727645, 'CREDIT_INCOME_PERCENT': 0.1355584126403995}


Normalisation des données

In [14]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

def normalize_train_test(train_data, test_data, save_path=model_folder):
    """
    Normalise les données d'entraînement et de test en ajustant le scaler sur les données d'entraînement
    et enregistre les paramètres de standardisation pour réutilisation dans le pipeline d'inférence.

    :param train_data: DataFrame d'entraînement
    :param test_data: DataFrame de test
    :param save_path: Chemin où enregistrer le scaler et les informations de normalisation
    :return: DataFrames normalisés (train_features_scaled, test_features_scaled)
    """

    # Séparer la colonne TARGET des données d'entraînement
    train_target = train_data['TARGET']
    train_features = train_data.drop(columns=['TARGET'])

    # Initialiser le scaler
    scaler = StandardScaler()

    # Ajuster le scaler sur les données d'entraînement et transformer les données d'entraînement
    train_features_scaled = scaler.fit_transform(train_features)

    # Transformer les données de test
    test_features_scaled = scaler.transform(test_data)

    # Convertir les données normalisées en DataFrame
    train_features_scaled = pd.DataFrame(train_features_scaled, columns=train_features.columns)
    test_features_scaled = pd.DataFrame(test_features_scaled, columns=test_data.columns)

    # Ajouter la colonne TARGET aux données d'entraînement normalisées
    train_features_scaled['TARGET'] = train_target.values

    # Sauvegarder le scaler pour réutilisation
    scaler_path = f'{save_path}/scaler.pkl'
    joblib.dump(scaler, scaler_path)
    print(f"Scaler sauvegardé à : {scaler_path}")

    return train_features_scaled, test_features_scaled





In [15]:
app_train , app_test = normalize_train_test(train_data,test_data, save_path=model_folder)

Scaler sauvegardé à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20240917_193333/scaler.pkl


Entrainement du modèle

In [16]:
categorical_features = app_train.select_dtypes(include=['object', 'category']).columns.tolist()
print('List of categorical features:', categorical_features)

List of categorical features: []


In [17]:

def catboost_model_with_mlflow_optimise(features, test_features, cat_features, n_folds=5, save_path=model_folder):
    """
    Entraîne et teste un modèle CatBoost en utilisant la validation croisée avec suivi MLflow et optimisation Optuna.
    Enregistre le modèle, les importances des caractéristiques et les métriques dans un dossier.

    Paramètres
    ----------
    features (pd.DataFrame): DataFrame des caractéristiques d'entraînement avec la colonne TARGET.
    test_features (pd.DataFrame): DataFrame des caractéristiques de test.
    cat_features (list): Liste des noms des caractéristiques catégorielles.
    n_folds (int, optionnel): Nombre de plis à utiliser pour la validation croisée (défaut: 5).
    save_path (str, optionnel): Dossier dans lequel enregistrer le modèle et les informations.

    Retourne
    -------
    submission (pd.DataFrame): Prédictions sur les données de test.
    feature_importances (pd.DataFrame): Importances des caractéristiques du modèle.
    metrics (pd.DataFrame): Métriques AUC de validation et d'entraînement pour chaque pli.
    """

    # Créer le dossier de sauvegarde s'il n'existe pas
    os.makedirs(save_path, exist_ok=True)

    # Extraire les identifiants
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']

    # Extraire les étiquettes pour l'entraînement
    labels = features['TARGET']

    # Supprimer les identifiants et la cible
    features = features.drop(columns=['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns=['SK_ID_CURR'])

    # Remplacer les valeurs NaN dans les caractéristiques catégorielles par 'missing'
    for cat_feature in cat_features:
        features[cat_feature].fillna('missing', inplace=True)
        test_features[cat_feature].fillna('missing', inplace=True)

    def objective(trial):
        """Fonction objectif pour l'optimisation Optuna."""
        model = CatBoostClassifier(
            iterations=trial.suggest_int('iterations', 5000, 10000),
            learning_rate=trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            depth=trial.suggest_int('depth', 4, 10),
            l2_leaf_reg=trial.suggest_loguniform('l2_leaf_reg', 1, 10),
            loss_function='Logloss',
            eval_metric='AUC',
            random_seed=50,
            cat_features=cat_features,
            logging_level='Silent',
            early_stopping_rounds=100,
            task_type='GPU' if get_gpu_device_count() > 0 else 'CPU'
        )

        out_of_fold = np.zeros(features.shape[0])
        valid_scores = []
        train_scores = []

        k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=50)

        for fold, (train_indices, valid_indices) in enumerate(k_fold.split(features)):
            print(f"Entraînement du pli {fold+1}/{n_folds}...")

            train_features, train_labels = features.iloc[train_indices], labels.iloc[train_indices]
            valid_features, valid_labels = features.iloc[valid_indices], labels.iloc[valid_indices]

            model.fit(
                train_features, train_labels,
                eval_set=(valid_features, valid_labels),
                use_best_model=True
            )

            out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]

            valid_auc = roc_auc_score(valid_labels, out_of_fold[valid_indices])
            train_auc = roc_auc_score(train_labels, model.predict_proba(train_features)[:, 1])
            valid_scores.append(valid_auc)
            train_scores.append(train_auc)

        overall_valid_auc = roc_auc_score(labels, out_of_fold)
        mean_train_auc = np.mean(train_scores)

        return overall_valid_auc

    # Exécuter l'optimisation Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=10)

    # Meilleurs paramètres trouvés
    best_params = study.best_params
    print(f"Meilleurs paramètres trouvés : {best_params}")

    # Entraîner le modèle final avec les meilleurs paramètres
    model = CatBoostClassifier(
        **best_params,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=50,
        cat_features=cat_features,
        logging_level='Silent',
        early_stopping_rounds=100,
        task_type='GPU' if get_gpu_device_count() > 0 else 'CPU'
    )

    out_of_fold = np.zeros(features.shape[0])
    valid_scores = []
    train_scores = []
    feature_importances = pd.DataFrame()

    k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=50)

    for fold, (train_indices, valid_indices) in enumerate(k_fold.split(features)):
        print(f"Entraînement du pli {fold+1}/{n_folds}...")

        train_features, train_labels = features.iloc[train_indices], labels.iloc[train_indices]
        valid_features, valid_labels = features.iloc[valid_indices], labels.iloc[valid_indices]

        model.fit(
            train_features, train_labels,
            eval_set=(valid_features, valid_labels),
            use_best_model=True
        )

        out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]

        valid_auc = roc_auc_score(valid_labels, out_of_fold[valid_indices])
        train_auc = roc_auc_score(train_labels, model.predict_proba(train_features)[:, 1])
        valid_scores.append(valid_auc)
        train_scores.append(train_auc)

        # Stocker les importances des caractéristiques
        fold_importances = pd.DataFrame()
        fold_importances['feature'] = features.columns
        fold_importances['importance'] = model.get_feature_importance()
        fold_importances['fold'] = fold + 1
        feature_importances = pd.concat([feature_importances, fold_importances], axis=0)

    overall_valid_auc = roc_auc_score(labels, out_of_fold)
    mean_train_auc = np.mean(train_scores)

    test_predictions = model.predict_proba(test_features)[:, 1]
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})

    metrics = pd.DataFrame({
        'fold': list(range(1, n_folds + 1)),
        'train_auc': train_scores,
        'valid_auc': valid_scores
    })

    # Sauvegarder le modèle, les FI et les métriques
    model_path = f'{save_path}/best_model.cbm'
    model.save_model(model_path)
    print(f"Modèle CatBoost sauvegardé à : {model_path}")

    feature_importances_path = f'{save_path}/feature_importances.csv'
    feature_importances.to_csv(feature_importances_path, index=False)
    print(f"Importances des caractéristiques sauvegardées à : {feature_importances_path}")

    metrics_path = f'{save_path}/metrics.csv'
    metrics.to_csv(metrics_path, index=False)
    print(f"Métriques sauvegardées à : {metrics_path}")

    return submission, feature_importances, metrics





In [18]:
import catboost
from catboost.utils import get_gpu_device_count


In [19]:
submission, fi, metrics = catboost_model_with_mlflow_optimise(app_train, app_test, categorical_features, n_folds=5, save_path=model_folder)
print('Baseline metrics')
print(metrics)

[I 2024-09-17 19:34:04,363] A new study created in memory with name: no-name-84f1ef7b-dc8a-4ad6-9d83-58f08160861e


Entraînement du pli 1/5...


Default metric period is 5 because AUC is/are not implemented for GPU


Entraînement du pli 2/5...


Default metric period is 5 because AUC is/are not implemented for GPU


...

Entraînement du pli 4/5...


Default metric period is 5 because AUC is/are not implemented for GPU


Entraînement du pli 5/5...


Default metric period is 5 because AUC is/are not implemented for GPU


Modèle CatBoost sauvegardé à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20240917_193333/best_model.cbm
Importances des caractéristiques sauvegardées à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20240917_193333/feature_importances.csv
Métriques sauvegardées à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20240917_193333/metrics.csv
Baseline metrics
   fold  train_auc  valid_auc
0     1   0.797377   0.764254
1     2   0.805616   0.767005
2     3   0.792345   0.770836
3     4   0.794899   0.766753
4     5   0.794915   0.767208


In [20]:
fi

,feature,importance,fold
0,NAME_CONTRACT_TYPE,0.832038,1
1,FLAG_OWN_CAR,0.489192,1
2,FLAG_OWN_REALTY,0.144370,1
3,CNT_CHILDREN,0.154767,1
4,AMT_INCOME_TOTAL,0.814430,1
...,...,...,...
238,WALLSMATERIAL_MODE_Panel,0.035243,5
239,"WALLSMATERIAL_MODE_Stone, brick",0.122203,5
240,WALLSMATERIAL_MODE_Wooden,0.027773,5
241,EMERGENCYSTATE_MODE_No,0.008467,5


In [21]:
import os
import subprocess
from datetime import datetime
import shutil  # Pour la copie des dossiers

# 1. Monter Google Drive (si ce n'est pas déjà fait)
from google.colab import drive
drive.mount('/content/drive')

# 2. Cloner le dépôt GitHub dans Google Colab
repo_url = "https://YOUR_TOKEN@github.com/bourydamien/projet_scoring.git"
repo_name = "projet_scoring"
branch = "main"  # Branche par défaut

# Remplacer `YOUR_TOKEN` par ton token GitHub personnel
repo_url = repo_url.replace("YOUR_TOKEN", "ton_token_github")  # Mets ici ton token GitHub

# Cloner le dépôt s'il n'est pas déjà cloné
if not os.path.exists(repo_name):
    subprocess.run(["git", "clone", repo_url])
    print(f"Dépôt cloné dans {repo_name}")

# 3. Déterminer le nom du dossier le plus récent dans `scoring_folder`
scoring_folder = '/content/drive/MyDrive/Colab Notebooks/data_scoring'
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_folder = f'{scoring_folder}/model_{timestamp}'

# Trouver le dossier le plus récent
folders = [f for f in os.listdir(scoring_folder) if os.path.isdir(os.path.join(scoring_folder, f))]
model_folders = [f for f in folders if f.startswith('model_')]

if model_folders:
    latest_folder = max(model_folders, key=lambda f: os.path.getctime(os.path.join(scoring_folder, f)))
    model_folder_path = os.path.join(scoring_folder, latest_folder)
else:
    raise FileNotFoundError("Aucun dossier modèle trouvé dans `scoring_folder`.")

print(f"Dossier modèle trouvé : {model_folder_path}")

# 4. Créer un dossier 'modeles' dans le dépôt s'il n'existe pas déjà
destination_folder = f'{repo_name}/modeles'
os.makedirs(destination_folder, exist_ok=True)

# 5. Copier tout le contenu du dossier `model_folder_path` vers `modeles` dans le dépôt
# Utiliser shutil pour copier tout le contenu
shutil.copytree(model_folder_path, os.path.join(destination_folder, latest_folder), dirs_exist_ok=True)

print(f"Fichiers copiés de {model_folder_path} vers {os.path.join(destination_folder, latest_folder)}")

# 6. Ajouter, committer et pousser les fichiers vers GitHub
def push_to_github(commit_message, repo_name, branch='main'):
    """
    Pousse les fichiers sur GitHub via Git après les avoir ajoutés et committés.

    :param commit_message: Message de commit Git
    :param repo_name: Nom du répertoire local Git
    :param branch: Branche du dépôt GitHub vers laquelle pousser (par défaut: 'main')
    """
    # Se déplacer dans le répertoire du dépôt
    os.chdir(repo_name)

    try:
        # Ajouter les fichiers au dépôt
        subprocess.run(["git", "add", "."], check=True)

        # Committer les fichiers avec un message
        subprocess.run(["git", "commit", "-m", commit_message], check=True)

        # Pousser les changements sur la branche spécifiée
        subprocess.run(["git", "push", "origin", branch], check=True)

        print("Les fichiers ont été poussés avec succès sur GitHub.")

    except subprocess.CalledProcessError as e:
        print(f"Une erreur s'est produite lors de l'exécution de la commande Git : {e}")

    # Revenir au répertoire principal
    os.chdir('..')

# 7. Appeler la fonction pour pousser sur GitHub
push_to_github("Ajout du dossier modèle avec les fichiers de scoring", repo_name, branch)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dépôt cloné dans projet_scoring
Dossier modèle trouvé : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20240917_080839
Fichiers copiés de /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20240917_080839 vers projet_scoring/modeles/model_20240917_080839
Une erreur s'est produite lors de l'exécution de la commande Git : Command '['git', 'commit', '-m', 'Ajout du dossier modèle avec les fichiers de scoring']' returned non-zero exit status 128.


In [22]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Sep 17 19:54:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              49W / 400W |    425MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--